In [1]:
'''

Given two integers dividend and divisor, divide two integers without using multiplication, division, and mod operator.

The integer division should truncate toward zero, which means losing its fractional part. For example, 8.345 would be truncated to 8, and -2.7335 would be truncated to -2.

Return the quotient after dividing dividend by divisor.

Note: Assume we are dealing with an environment that could only store integers within the 32-bit signed integer range: [−231, 231 − 1]. For this problem, if the quotient is strictly greater than 231 - 1, then return 231 - 1, and if the quotient is strictly less than -231, then return -231.

 

Example 1:

Input: dividend = 10, divisor = 3
Output: 3
Explanation: 10/3 = 3.33333.. which is truncated to 3.
Example 2:

Input: dividend = 7, divisor = -3
Output: -2
Explanation: 7/-3 = -2.33333.. which is truncated to -2.
 

Constraints:

-231 <= dividend, divisor <= 231 - 1
divisor != 0

TIP 1:
    1. Solution 1
        a. sign for two numbers = -1 if a < 0 ^ b < 0 else 1
        b. Use typical subtract until less than divisor.
        c. O(a/b)
    3. Solution 2
        a. find quotient using bit tricks
        b. 

'''

class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        if dividend == 0:
            return 0
        sign = -1 if((dividend < 0) ^ (divisor < 0)) else 1
        ans = 0
        num  = abs(dividend)
        base = abs(divisor)
        while num >= base:
            counter = 1
            decr    = base
            while num >= decr:
                num -= decr
                ans += counter
                counter += counter
                decr += decr
        ans = ans if sign > 0 else ~(ans) + 1
        return min(max(-2147483648, ans), 2147483647)


# TLE
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        sign= 1 if (dividend > 0 and divisor >0) or (dividend < 0 and divisor < 0) else -1
        num = abs(dividend)
        divisor = abs(divisor)
        count = 0
        while num >= divisor:
            num -= divisor
            count += 1
        return sign * count

In [2]:
# Tracking quotient --- wrong answer, off by 1 for -ve input
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        sign = -1 if((dividend < 0) ^ (divisor < 0)) else 1
        dividend = abs(dividend)
        divisor = abs(divisor)
        quotient = 0
        temp = 0
        for i in range(31, -1, -1):
            if (temp + (divisor << i) <= dividend):
                temp += divisor << i
                quotient |= 1 << i
        if sign ==-1 :
            quotient=-quotient
        return quotient

In [26]:
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        sign = -1 if (dividend >= 0 and divisor < 0) or (dividend < 0 and divisor >= 0) else 1
        dividend = abs(dividend)
        divisor = abs(divisor)
        result = len(range(0, dividend-divisor+1, divisor))
        if sign == -1:
            result = -result
        minus_limit = -(2**31)
        plus_limit = (2**31 - 1)
        result = min(max(result, minus_limit), plus_limit)
        return result

In [27]:
class Solution:
    def divide(self, dividend, divisor):
        positive = (dividend < 0) is (divisor < 0)
        dividend, divisor = abs(dividend), abs(divisor)
        res = 0
        while dividend >= divisor:
            temp, i = divisor, 1
            while dividend >= temp:
                dividend -= temp
                res += i
                i <<= 1
                temp <<= 1
        if not positive:
            res = -res
        return min(max(-2147483648, res), 2147483647)

In [28]:
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        a = abs(dividend)
        b=abs(divisor)
        
        negative = (dividend<0 and divisor>=0) or (dividend>=0 and divisor<0)
        
        output = 0
        
        while a>=b:
            counter = 1
            decrement = b
            
            while a>=decrement:
                a-=decrement
                
                output+=counter
                counter+=counter
                decrement+=decrement
                
        output = output if not negative else -output
        
        return min(max(-2147483648, output), 2147483647)

In [29]:
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        is_negative = (dividend < 0) != (divisor < 0)
        divisor, dividend = abs(divisor), abs(dividend)

        quotient = 0
        the_sum = divisor

        while the_sum <= dividend:
            current_quotient = 1
            while (the_sum << 1) <= dividend:
                the_sum <<= 1
                current_quotient <<= 1            
            dividend -= the_sum
            the_sum = divisor
            quotient += current_quotient

        return min(2147483647, max(-quotient if is_negative else quotient, -2147483648))
    
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        is_negative = (dividend < 0) != (divisor < 0)
        divisor, dividend = abs(divisor), abs(dividend)

        quotient = 0
        the_sum = divisor

        while the_sum <= dividend:
            current_quotient = 1
            while (the_sum + the_sum) <= dividend:
                the_sum += the_sum
                current_quotient += current_quotient
            dividend -= the_sum
            the_sum = divisor
            quotient += current_quotient

        return min(2147483647, max(-quotient if is_negative else quotient, -2147483648))

In [30]:
# https://leetcode.com/problems/divide-two-integers/solutions/1084819/python-o-log-n-honest-solution-explained/

# I do not really like these type of problems, where you restricted in using operations. What we can use if we can not use multiplications and divisions: we can use only addition and subtraction. Let consider an example:
# 100 // 7

# We can try just to subtract 7 while it is possible. However potentially it can be quite long, if dividend is big and divisor is small. Let us multiply 7 by 2 (in fact it is not multiplication, but addition with itself), until we are smaller than 100. 7 -> 14 -> 28 -> 56. Ans subtract 56 now, so we have 8 as result and we need to divide 44 by 7 now. Repeat procedure, so we subtract 28, and we have 8 + 4 as result and 44 - 28 = 16 = 2. Finally, we subtract 14 and we have 8 + 4 + 2 = 14 as result.

# Let us precalculate pairs (7, 1), (14, 2), (28, 4), (56, 8). Then we iterate through these pairs in opposite direction and if we can subtract corresponding number, we subtract, if not - we go to the next one. Also we need to deal with signs and overflows here.

# Complexity: time complexity is O(log n), where n = divident/divisor: there will be O(log n) terms in our cand list as well as this is limit for number of steps. Space complexity is O(log n) as well. What I mean by honest here, that a lot of people here in discussion either use some tricks which are not allowed, or complexity is wrong.

class Solution:
    def divide(self, dividend, divisor):
        if dividend == -1<<31 and divisor == -1: return (1<<31)-1

        a, b = abs(dividend), abs(divisor)
        sign = (dividend < 0) == (divisor < 0)
        res, cand = 0, [(1, b)]
        
        while b << 1 <= a:
            cand += [(cand[-1][0]<<1, b<<1)]
            b <<= 1
            
        for pw, num in cand[::-1]:
            if a >= num:
                a, res = a - num, res + pw
                
        return res if sign else -res

In [40]:
# Wrong
from math import log, e
class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        if dividend == 0:
            return 0
        sign = -1 if((dividend < 0) ^ (divisor < 0)) else 1
        ans = None
        if abs(divisor) == 1:
            ans = abs(dividend) * sign
        else:
            num  = abs(dividend)
            base = abs(divisor)
            ans  = e ** (log(num) - log(base))
            if (round(ans) - ans) < 0.0000000001:
                ans = round(ans)
            ans  = int(ans)*sign
        return min(max(-2147483648, ans), 2147483647)